In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
import sparknlp
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[4]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5")\
    .config("spark.kryoserializer.buffer.max", "1G")\
    .getOrCreate()
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.functions import from_unixtime, to_date, year, udf, explode, split, col, length, rank, dense_rank, avg, sum
from pyspark.sql.window import Window

In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [4]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
from wordcloud import WordCloud
from textblob import TextBlob

In [5]:
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
data = spark.read.csv('C:/Users/User/cs226/Geo_Tagged_Tweets_State_Info/dataset_repub_election_day_geo_tagged_with_State.csv')
data.head()

Row(_c0=None, _c1='Datetime', _c2='Tweet Id', _c3='Text', _c4='Username', _c5='User_Location', _c6='coordinate_longitude', _c7='coordinate_latitude', _c8='place_Fullname', _c9='place_name', _c10='State_Name', _c11='State_Code', _c12='place_country', _c13='countryCode')

In [18]:
newdata=data
#[["created_at","text","geo.full_name","geo.geo.bbox","geo.name"]]
#newdata.isnull().sum().sort_values(ascending=False)
#newdata=newdata.dropna()
newdata.head()

Row(_c0=None, _c1='Datetime', _c2='Tweet Id', _c3='Text', _c4='Username', _c5='User_Location', _c6='coordinate_longitude', _c7='coordinate_latitude', _c8='place_Fullname', _c9='place_name', _c10='State_Name', _c11='State_Code', _c12='place_country', _c13='countryCode')

In [27]:
newdata = pd.read_csv('C:/Users/User/cs226/Geo_Tagged_Tweets_State_Info/dataset_repub_election_day_geo_tagged_with_State.csv')
#data.head()
text10k=[]
for ttext in newdata['Text'].apply(str):
    Word_Split = []
    for word in  re.sub("\W"," ",ttext ).split():
        Word_Split.append(word)
    text10k.append(' '.join(Word_Split))
newdata["Word_Split"]= text10k
newdata.head()

Unnamed: 0                   Datetime             Tweet Id  \
0           0  2022-11-08 23:59:51+00:00  1590132021032267776   
1           1  2022-11-08 23:59:29+00:00  1590131926169714688   
2           2  2022-11-08 23:59:28+00:00  1590131921753108480   
3           3  2022-11-08 23:59:25+00:00  1590131911103746050   
4           4  2022-11-08 23:59:08+00:00  1590131838248701952   

                                                Text       Username  \
0  @LibertyAnders I agree. Democrats keep pushing...      johnwallz   
1  Eating fried green tomatoes at Brad Raffensper...       csumnall   
2  @TiborPNagyJr Vote Republicans in senatorial r...  YeGodanaNewte   
3  Biden should of never payed off any student de...     geis_shane   
4  I’m a #habitualvoter. The driving force behind...        amybree   

  User_Location  coordinate_longitude  coordinate_latitude     place_Fullname  \
0   Amherst, NH            -71.638615            42.789980        Amherst, NH   
1           NaN            -84.468784            33.889334  Garden & Gun Club   
2  Flushing, NY            -73.962582            40.541722         Queens, NY   
3           NaN            -78.371861            40.514563      Greenwood, PA   
4      Maryland            -79.487651            37.886607      Maryland, USA   

          place_name     State_Name State_Code  place_country countryCode  \
0            Amherst  New Hampshire         NH  United States          US   
1  Garden & Gun Club            NaN        NaN  United States          US   
2             Queens       New York         NY  United States          US   
3          Greenwood   Pennsylvania         PA  United States          US   
4           Maryland       Maryland         MD  United States          US   

                                          Word_Split  
0  LibertyAnders I agree Democrats keep pushing t...  
1  Eating fried green tomatoes at Brad Raffensper...  
2  TiborPNagyJr Vote Republicans in senatorial ra...  
3  Biden should of never payed off any student de...  
4  I m a habitualvoter The driving force behind m...

In [28]:
sentiment_result = []
for words in newdata["Word_Split"]:
    Tweet = TextBlob(words)
    sentiment_value = Tweet.sentiment[0]
    if (sentiment_value > 0):
        sentiment_result.append('Positive')
    elif (sentiment_value < 0):
        sentiment_result.append('Negative')
    else:
        sentiment_result.append('Neutral')   

In [29]:
newdata["sentiment"]=sentiment_result
newdata.head()

Unnamed: 0                   Datetime             Tweet Id  \
0           0  2022-11-08 23:59:51+00:00  1590132021032267776   
1           1  2022-11-08 23:59:29+00:00  1590131926169714688   
2           2  2022-11-08 23:59:28+00:00  1590131921753108480   
3           3  2022-11-08 23:59:25+00:00  1590131911103746050   
4           4  2022-11-08 23:59:08+00:00  1590131838248701952   

                                                Text       Username  \
0  @LibertyAnders I agree. Democrats keep pushing...      johnwallz   
1  Eating fried green tomatoes at Brad Raffensper...       csumnall   
2  @TiborPNagyJr Vote Republicans in senatorial r...  YeGodanaNewte   
3  Biden should of never payed off any student de...     geis_shane   
4  I’m a #habitualvoter. The driving force behind...        amybree   

  User_Location  coordinate_longitude  coordinate_latitude     place_Fullname  \
0   Amherst, NH            -71.638615            42.789980        Amherst, NH   
1           NaN            -84.468784            33.889334  Garden & Gun Club   
2  Flushing, NY            -73.962582            40.541722         Queens, NY   
3           NaN            -78.371861            40.514563      Greenwood, PA   
4      Maryland            -79.487651            37.886607      Maryland, USA   

          place_name     State_Name State_Code  place_country countryCode  \
0            Amherst  New Hampshire         NH  United States          US   
1  Garden & Gun Club            NaN        NaN  United States          US   
2             Queens       New York         NY  United States          US   
3          Greenwood   Pennsylvania         PA  United States          US   
4           Maryland       Maryland         MD  United States          US   

                                          Word_Split sentiment  
0  LibertyAnders I agree Democrats keep pushing t...  Negative  
1  Eating fried green tomatoes at Brad Raffensper...  Negative  
2  TiborPNagyJr Vote Republicans in senatorial ra...  Negative  
3  Biden should of never payed off any student de...  Positive  
4  I m a habitualvoter The driving force behind m...  Positive

In [30]:
newdata=newdata[["Datetime","Tweet Id","Text","Username","User_Location","coordinate_longitude","coordinate_latitude","place_Fullname","place_name","State_Name","State_Code","place_country","countryCode","sentiment"]]

In [31]:
newdata.to_csv('C:/Users/User/cs226/Geo_Tagged_Tweets_State_Info/sentiment/sentiment_repub_election_day_geo_tagged_with_State.csv')